# Wege Inland

## 1. Import

### 1.1 Bib

In [1]:
import pandas as pd
from folium.plugins import HeatMap
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import LineString

import folium
from folium import PolyLine
from IPython.display import IFrame

from ipywidgets import interact

### 1.2 Data

In [2]:
# Datei einlesen
df_wege = pd.read_csv('/Users/orianagwerder/ANM_Gruppe_Y/Data/wege.csv', encoding='latin1', sep=';')

In [3]:
df_wege.head()

,HHNR,WP,AUSNR,WEGNR,DMOD,f51100,f51400,f51100time,f51400time,pseudo,...,w_luftlin,w_rdist,wmittel1,wmittel1a,wmittel2,wmittel2a,wzweck1,wzweck2,wzweck3,w_dist_obj2
0,110001,1.606924,1,1,2,555,565,9:15:00,9:25:00,1,...,0.333127,0.413,8,2,8,2,5.0,1,5,0.333127
1,110001,1.606924,1,2,2,575,580,9:35:00,9:40:00,1,...,0.863416,1.483,8,2,8,2,4.0,1,4,0.863416
2,110001,1.606924,1,3,2,610,620,10:10:00,10:20:00,1,...,1.123565,1.407,8,2,8,2,4.0,2,4,1.123565
3,110001,1.606924,2,4,2,750,760,12:30:00,12:40:00,1,...,2.167349,3.973,8,2,8,2,8.0,1,8,2.167349
4,110001,1.606924,2,5,2,810,825,13:30:00,13:45:00,1,...,3.127501,4.811,8,2,8,2,5.0,1,5,3.127501


In [4]:
df_wege.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151188 entries, 0 to 151187
Data columns (total 75 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   HHNR                 151188 non-null  int64  
 1   WP                   151188 non-null  float64
 2   AUSNR                151188 non-null  int64  
 3   WEGNR                151188 non-null  int64  
 4   DMOD                 151188 non-null  int64  
 5   f51100               151188 non-null  int64  
 6   f51400               151188 non-null  int64  
 7   f51100time           151188 non-null  object 
 8   f51400time           151188 non-null  object 
 9   pseudo               151188 non-null  int64  
 10  S_X                  151188 non-null  float64
 11  S_Y                  151188 non-null  float64
 12  S_QAL                151188 non-null  int64  
 13  S_BFS                151188 non-null  int64  
 14  S_PLZ                151188 non-null  int64  
 15  S_Ort            

## 2. Clear Data

In [5]:
df_wege = df_wege.dropna()


In [6]:
df_wege.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127531 entries, 0 to 151187
Data columns (total 75 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   HHNR                 127531 non-null  int64  
 1   WP                   127531 non-null  float64
 2   AUSNR                127531 non-null  int64  
 3   WEGNR                127531 non-null  int64  
 4   DMOD                 127531 non-null  int64  
 5   f51100               127531 non-null  int64  
 6   f51400               127531 non-null  int64  
 7   f51100time           127531 non-null  object 
 8   f51400time           127531 non-null  object 
 9   pseudo               127531 non-null  int64  
 10  S_X                  127531 non-null  float64
 11  S_Y                  127531 non-null  float64
 12  S_QAL                127531 non-null  int64  
 13  S_BFS                127531 non-null  int64  
 14  S_PLZ                127531 non-null  int64  
 15  S_Ort            

## 2. Visualisierung

In [7]:
# Funktion zum Erstellen der Karte mit Filter
def create_map(selected_kanton):
    # Filtern der Daten basierend auf dem ausgewählten Kanton im ursprünglichen DataFrame
    filtered_df = df_wege[df_wege['S_KANTON'] == selected_kanton]
    
    # Routen erstellen und zählen
    filtered_df['route'] = list(zip(filtered_df['S_X'], filtered_df['S_Y'], filtered_df['Z_X'], filtered_df['Z_Y']))
    route_counts = filtered_df['route'].value_counts().reset_index()
    route_counts.columns = ['route', 'count']
    
    # Karte auf die Schweiz zentrieren und passenden Zoom setzen
    m = folium.Map(location=[46.8182, 8.2275], zoom_start=7)
    
    # Routen zur Karte hinzufügen
    for _, row in route_counts.iterrows():
        route = row['route']
        count = row['count']
        start = (route[1], route[0])  # (Latitude, Longitude)
        end = (route[3], route[2])    # (Latitude, Longitude)
        
        # Linie hinzufügen mit Dicke und Transparenz basierend auf der Anzahl der Vorkommen
        folium.PolyLine(
            locations=[start, end],
            weight=2 + (count ** 0.5),  # Wurzel für eine dynamische Gewichtung
            color='blue',
            opacity=0.3  # Transparenz erhöhen
        ).add_to(m)
    
    # Karte anzeigen
    return m

# Interaktive Auswahl für den Kanton
kantone = df_wege['S_KANTON'].unique()
interact(create_map, selected_kanton=kantone)


interactive(children=(Dropdown(description='selected_kanton', options=(22, 17, 2, 14, 11, 25, 7, 13, 12, 21, 1…

<function __main__.create_map(selected_kanton)>

In [8]:
# Route erstellen und zählen
df_wege['route'] = list(zip(df_wege['S_X'], df_wege['S_Y'], df_wege['Z_X'], df_wege['Z_Y']))
route_counts = df_wege['route'].value_counts().reset_index()
route_counts.columns = ['route', 'count']

# Ausgabe der häufigsten Routen
print("Häufigkeit der einzelnen Routen:")
print(route_counts.head(10))  # Die Top-10-Routen anzeigen

# Alternativ den gesamten DataFrame anzeigen
route_counts_sorted = route_counts.sort_values(by='count', ascending=False)
route_counts_sorted.reset_index(drop=True, inplace=True)
print(route_counts_sorted)


Häufigkeit der einzelnen Routen:
                                               route  count
0  (8.6502008747, 47.477272979, 8.6502008747, 47....     10
1  (7.4258466778, 47.199596107, 7.4258466778, 47....      9
2  (9.4265302431, 47.201367283, 9.4265302431, 47....      7
3  (6.8387239877, 46.461473384, 6.8387239877, 46....      6
4  (7.3095288616, 46.204865409, 7.3095288616, 46....      6
5  (8.5785685388, 47.734966098, 8.5785685388, 47....      6
6  (6.8213320101, 47.099326918, 6.8213320101, 47....      6
7  (9.0431366378, 47.461371192, 9.0431366378, 47....      6
8  (9.3524912492, 47.548868457, 9.3524912492, 47....      6
9  (7.159775895, 47.105349282, 7.159775895, 47.10...      6
                                                    route  count
0       (8.6502008747, 47.477272979, 8.6502008747, 47....     10
1       (7.4258466778, 47.199596107, 7.4258466778, 47....      9
2       (9.4265302431, 47.201367283, 9.4265302431, 47....      7
3       (7.159775895, 47.105349282, 7.159775895

In [9]:
# Höchster Wert der Nutzung (maximale Anzahl der Routen-Nutzung)
max_count = route_counts['count'].max()

# Ausgabe des höchsten Wertes und der zugehörigen Route(n)
most_frequent_routes = route_counts[route_counts['count'] == max_count]

print("Höchste Nutzungshäufigkeit für eine Route:", max_count)
print("Routen mit dieser Häufigkeit:")
print(most_frequent_routes)


Höchste Nutzungshäufigkeit für eine Route: 10
Routen mit dieser Häufigkeit:
                                               route  count
0  (8.6502008747, 47.477272979, 8.6502008747, 47....     10
